In [1]:
import csv
import numpy as np

import sys

In [2]:
def extractfeaturesnaive(row, B):
    v = np.zeros(B)
    twits = row.split()
    for twit in twits:
        if "http" not in twit:
            v[hash(twit.lower()) % B] = 1
    return v

def extractfeaturesxnaive(row, B):
    v = np.zeros(B)
    i = 0   
    if "Hillary's" in row:
        i = -1
    if "Crooked Hillary" in row:
        i = 1 
    if "#Crooked" in row:
        i = -1   
    if "@CNN" in row:
        i = 1        
    if "@realDonaldTrump" in row:
        i = 1
    if "http" in row:
            i = -1 
    if '"@' in row:
        i = 1
    if "all of the" in row or "badly" in row or "angry" in row or "weak" in row or "Cruz" in row or "crazy" in row or "said" in row or "hate" in row:
        i = 1
    if "Justice" in row and row.index("Justice")!= 0:
        i = -1
    twits = row.split()
    for twit in twits:
        if "http" not in twit:
            v[hash(twit.lower()) % B] = 1 
          
    if i == -1:
        v = np.zeros(B)
    if i == 1:
        v = np.ones(B)   

    return v

def importData(filename,B):
    with open(filename, newline='') as csvfile:
        filereader = csv.reader(csvfile, quotechar='|')
        i = 0
        xdata = []
        ydata = []
        for row in filereader:
            if i != 0:
                xdata.append(extractfeaturesnaive(row[1],B))
                ydata.append(int(row[17]))
            i = 1
        xdata = np.asarray(xdata)
        ydata = np.asarray(ydata)
        return xdata,ydata 
    
def importXData(filename,B):
    with open(filename, newline='') as csvfile:
        filereader = csv.reader(csvfile, quotechar='|')
        i = 0
        xdata = []
        for row in filereader:
            if i != 0:
                xdata.append(extractfeaturesxnaive(row[1],B))
            i = 1
        xdata = np.asarray(xdata)
        return xdata

In [3]:
def naivebayesPY(x,y):
    """
    naivebayesPY(x,y) returns [pos,neg]

    Computation of P(Y)
    Input:
        x : n input vectors of d dimensions (nxd)
        y : n labels (-1 or +1) (nx1)

    Output:
    pos: probability p(y=1)
    neg: probability p(y=-1)
    """
    
    # add one positive and negative example to avoid division by zero ("plus-one smoothing")
    y = np.concatenate([y, [-1,1]])
    n = len(y)
    ## fill in code here
    #raise NotImplementedError('Your code goes here!')
    pos = ( np.sum(y) + n) / 2 / n
    neg = 1 - pos
    return (pos, neg)

In [4]:
#<GRADED>
def naivebayesPY(x,y):
    """
    naivebayesPY(x,y) returns [pos,neg]

    Computation of P(Y)
    Input:
        x : n input vectors of d dimensions (nxd)
        y : n labels (-1 or +1) (nx1)

    Output:
    pos: probability p(y=1)
    neg: probability p(y=-1)
    """
    
    # add one positive and negative example to avoid division by zero ("plus-one smoothing")
    y = np.concatenate([y, [-1,1]])
    n = len(y)
    ## fill in code here
    #raise NotImplementedError('Your code goes here!')
    pos = ( np.sum(y) + n) / 2 / n
    neg = 1 - pos
    return (pos, neg)
#</GRADED>


In [5]:
def naivebayesPXY(x,y):
    """
    naivebayesPXY(x,y) returns [posprob,negprob]
    
    Input:
        x : n input vectors of d dimensions (nxd)
        y : n labels (-1 or +1) (nx1)
    
    Output:
    posprob: probability vector of p(x|y=1) (1xd)
    negprob: probability vector of p(x|y=-1) (1xd)
    """
    
    # add one positive and negative example to avoid division by zero ("plus-one smoothing")
    n, d = x.shape
    x = np.concatenate([x, np.ones((2,d))])
    y = np.concatenate([y, [-1,1]])
    n, d = x.shape
    ## fill in code here
    #raise NotImplementedError('Your code goes here!')
    Z=np.c_[x,y]
    Z=Z[np.lexsort(Z.T)]
    pos,neg=0,0
    for m in y:
        if m==-1:
            neg+=1
    Z=Z[:,:-1]
    negprob=Z[:neg].sum(axis=0)/(Z[:neg].sum())
    posprob=Z[neg:].sum(axis=0)/(Z[neg:].sum())
    return (posprob,negprob)


In [6]:
def naivebayes(x,y,xtest):
    """
    naivebayes(x,y) returns logratio 
    
    Computation of log P(Y|X=x1) using Bayes Rule
    Input:
    x : n input vectors of d dimensions (nxd)
    y : n labels (-1 or +1)
    xtest: input vector of d dimensions (1xd)
    
    Output:
    logratio: log (P(Y = 1|X=xtest)/P(Y=-1|X=xtest))
    """
    
    ## fill in code here
    #raise NotImplementedError('Your code goes here!')
    n, d = x.shape
    posprob, negprob=0,0
    for i in range(n):
        if y[i]==1 and (x[i]==xtest).all():
            posprob+=1
        if y[i]==-1 and (x[i]==xtest).all():
            negprob+=1      
    pos,neg =naivebayesPY(x,y)
    posprob= posprob 
    negprob= negprob 
    return np.log((posprob+1/n)/(negprob+1/n))
            

In [7]:
def naivebayesCL(x,y):
    """
    naivebayesCL(x,y) returns [w,b]
    Implementation of a Naive Bayes classifier
    Input:
    x : n input vectors of d dimensions (nxd)
    y : n labels (-1 or +1)

    Output:
    w : weight vector of d dimensions
    b : bias (scalar)
    """
    
    ## fill in code here
    #raise NotImplementedError('Your code goes here!')
    n,d=x.shape
    x1,x2=naivebayesPXY(x,y)
    y1,y2=naivebayesPY(x,y)
    w= np.zeros(d)
    for i in range(d):
        w[i]=np.log(x1[i]/x2[i])
    b = np.log(y1/y2)
    return (w,b)


In [8]:
def classifyLinear(x,w,b):
    """
    classifyLinear(x,w,b) returns preds
    
    Make predictions with a linear classifier
    Input:
    x : n input vectors of d dimensions (nxd)
    w : weight vector of d dimensions
    b : bias (optional)
    
    Output:
    preds: predictions
    """
    n,d = x.shape
    pred = np.sign(x.dot(w)+b)
    return pred


In [9]:
import random
B = int(random.random()*2000+2000)
print(B)
xTr,yTr = importData('train.csv',B)

w,b=naivebayesCL(xTr,yTr)

n = len(yTr)

xdata = [xTr[:int(n/10)],xTr[int(n/10):int(n/10*2)],xTr[int(n/10*2):int(n/10*3)],
         xTr[int(n/10*3):int(n/10*4)],xTr[int(n/10*4):int(n/10*5)],xTr[int(n/10*5):int(n/10*6)],
         xTr[int(n/10*6):int(n/10*7)],xTr[int(n/10*7):int(n/10*8)],xTr[int(n/10*8):int(n/10*9)],xTr[int(n/10*9):]]
ydata = [yTr[:int(n/10)],yTr[int(n/10):int(n/10*2)],yTr[int(n/10*2):int(n/10*3)],
         yTr[int(n/10*3):int(n/10*4)],yTr[int(n/10*4):int(n/10*5)],yTr[int(n/10*5):int(n/10*6)],
         yTr[int(n/10*6):int(n/10*7)],yTr[int(n/10*7):int(n/10*8)],yTr[int(n/10*8):int(n/10*9)],yTr[int(n/10*9):]]

xdata1 = np.concatenate((xdata[1],xdata[2],xdata[3],xdata[4],xdata[5],xdata[6],xdata[7],xdata[8],xdata[9]))
xdata2 = np.concatenate((xdata[0],xdata[2],xdata[3],xdata[4],xdata[5],xdata[6],xdata[7],xdata[8],xdata[9]))
xdata3 = np.concatenate((xdata[0],xdata[1],xdata[3],xdata[4],xdata[5],xdata[6],xdata[7],xdata[8],xdata[9]))
xdata4 = np.concatenate((xdata[0],xdata[1],xdata[2],xdata[4],xdata[5],xdata[6],xdata[7],xdata[8],xdata[9]))
xdata5 = np.concatenate((xdata[0],xdata[1],xdata[2],xdata[3],xdata[5],xdata[6],xdata[7],xdata[8],xdata[9]))
xdata6 = np.concatenate((xdata[0],xdata[1],xdata[2],xdata[3],xdata[4],xdata[6],xdata[7],xdata[8],xdata[9]))
xdata7 = np.concatenate((xdata[0],xdata[1],xdata[2],xdata[3],xdata[4],xdata[5],xdata[7],xdata[8],xdata[9]))
xdata8 = np.concatenate((xdata[0],xdata[1],xdata[2],xdata[3],xdata[4],xdata[5],xdata[6],xdata[8],xdata[9]))
xdata9 = np.concatenate((xdata[0],xdata[1],xdata[2],xdata[3],xdata[4],xdata[5],xdata[6],xdata[7],xdata[9]))
xdata10 = np.concatenate((xdata[0],xdata[1],xdata[2],xdata[3],xdata[4],xdata[5],xdata[6],xdata[7],xdata[8]))

ydata1 = np.concatenate((ydata[1],ydata[2],ydata[3],ydata[4],ydata[5],ydata[6],ydata[7],ydata[8],ydata[9]))
ydata2 = np.concatenate((ydata[0],ydata[2],ydata[3],ydata[4],ydata[5],ydata[6],ydata[7],ydata[8],ydata[9]))
ydata3 = np.concatenate((ydata[0],ydata[1],ydata[3],ydata[4],ydata[5],ydata[6],ydata[7],ydata[8],ydata[9]))
ydata4 = np.concatenate((ydata[0],ydata[1],ydata[2],ydata[4],ydata[5],ydata[6],ydata[7],ydata[8],ydata[9]))
ydata5 = np.concatenate((ydata[0],ydata[1],ydata[2],ydata[3],ydata[5],ydata[6],ydata[7],ydata[8],ydata[9]))
ydata6 = np.concatenate((ydata[0],ydata[1],ydata[2],ydata[3],ydata[4],ydata[6],ydata[7],ydata[8],ydata[9]))
ydata7 = np.concatenate((ydata[0],ydata[1],ydata[2],ydata[3],ydata[4],ydata[5],ydata[7],ydata[8],ydata[9]))
ydata8 = np.concatenate((ydata[0],ydata[1],ydata[2],ydata[3],ydata[4],ydata[5],ydata[6],ydata[8],ydata[9]))
ydata9 = np.concatenate((ydata[0],ydata[1],ydata[2],ydata[3],ydata[4],ydata[5],ydata[6],ydata[7],ydata[9]))
ydata10 = np.concatenate((ydata[0],ydata[1],ydata[2],ydata[3],ydata[4],ydata[5],ydata[6],ydata[7],ydata[8]))


w1,b1=naivebayesCL(xdata1,ydata1)
w2,b2=naivebayesCL(xdata2,ydata2)
w3,b3=naivebayesCL(xdata3,ydata3)
w4,b4=naivebayesCL(xdata4,ydata4)
w5,b5=naivebayesCL(xdata5,ydata5)
w6,b6=naivebayesCL(xdata6,ydata6)
w7,b7=naivebayesCL(xdata7,ydata7)
w8,b8=naivebayesCL(xdata8,ydata8)
w9,b9=naivebayesCL(xdata9,ydata9)
w10,b10=naivebayesCL(xdata10,ydata10)

print('Training error 1: %.2f%%' % (100 *(classifyLinear(xdata[0], w1, b1) != ydata[0]).mean()))
print('Training error 2: %.2f%%' % (100 *(classifyLinear(xdata[1], w2, b2) != ydata[1]).mean()))
print('Training error 3: %.2f%%' % (100 *(classifyLinear(xdata[2], w3, b3) != ydata[2]).mean()))
print('Training error 4: %.2f%%' % (100 *(classifyLinear(xdata[3], w4, b4) != ydata[3]).mean()))
print('Training error 5: %.2f%%' % (100 *(classifyLinear(xdata[4], w5, b5) != ydata[4]).mean()))
print('Training error 6: %.2f%%' % (100 *(classifyLinear(xdata[5], w6, b6) != ydata[5]).mean()))
print('Training error 7: %.2f%%' % (100 *(classifyLinear(xdata[6], w7, b7) != ydata[6]).mean()))
print('Training error 8: %.2f%%' % (100 *(classifyLinear(xdata[7], w8, b8) != ydata[7]).mean()))
print('Training error 9: %.2f%%' % (100 *(classifyLinear(xdata[8], w9, b9) != ydata[8]).mean()))
print('Training error 10: %.2f%%' % (100 *(classifyLinear(xdata[9], w10, b10) != ydata[9]).mean()))

xTe=importXData('test.csv',B)
yTe = np.sign(classifyLinear(xTe, w1, b1)+ classifyLinear(xTe, w2, b2)+classifyLinear(xTe, w3, b3)   \
              + classifyLinear(xTe, w4, b4)+ classifyLinear(xTe, w5, b5)+classifyLinear(xTe, w6, b6) \
              + classifyLinear(xTe, w7, b7)+ classifyLinear(xTe, w8, b8)+classifyLinear(xTe, w9, b9))


for i in range(len(yTe)):
    if (xTe[i]==np.zeros(B)).all():
        yTe[i] = -1
        
for i in range(len(yTe)):
    if (xTe[i]==np.ones(B)).all():
        yTe[i] = 1



3180
Training error 1: 24.07%
Training error 2: 24.77%
Training error 3: 22.02%
Training error 4: 14.68%
Training error 5: 20.18%
Training error 6: 13.76%
Training error 7: 21.10%
Training error 8: 20.18%
Training error 9: 17.43%
Training error 10: 31.19%


In [10]:
def output(yTe):
    n= len(yTe)
    with open('output.csv', 'w', newline='') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        spamwriter.writerow(('ID','Label'))
        for i in range(n):
            spamwriter.writerow((i ,int(yTe[i])))
    return 
output(yTe)